In [1]:
import openai
from dotenv import load_dotenv

load_dotenv()  # takes environment variables from .env

In [15]:
# Configure to use vLLM’s API server (base URL only – methods will pick the right endpoint)
openai.api_base = "https://api.openai.com/v1"

# Specify the chat model
model = "gpt-4o"

# Create a chat completion with token‐level logprobs
completion = openai.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "A robot may not injure a human being"}],
    stream=False,
    logprobs=True,  # Return top 5 most probable tokens
    top_logprobs=5  
)

print(completion)

ChatCompletion(id='chatcmpl-BT3zyGiOSDKxPo2hwEh6oimRIUNZx', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='The', bytes=[84, 104, 101], logprob=-0.8941613435745239, top_logprobs=[TopLogprob(token='or', bytes=[111, 114], logprob=-0.8941613435745239), TopLogprob(token='The', bytes=[84, 104, 101], logprob=-0.8941613435745239), TopLogprob(token="You're", bytes=[89, 111, 117, 39, 114, 101], logprob=-2.6441612243652344), TopLogprob(token='This', bytes=[84, 104, 105, 115], logprob=-3.3941612243652344), TopLogprob(token='You', bytes=[89, 111, 117], logprob=-3.3941612243652344)]), ChatCompletionTokenLogprob(token=' phrase', bytes=[32, 112, 104, 114, 97, 115, 101], logprob=-0.4791475534439087, top_logprobs=[TopLogprob(token=' phrase', bytes=[32, 112, 104, 114, 97, 115, 101], logprob=-0.4791475534439087), TopLogprob(token=' statement', bytes=[32, 115, 116, 97, 116, 101, 109, 101, 110, 116], logprob=-1.1041475534439087), TopLogprob(

In [17]:
completion.choices[0].logprobs.content


[ChatCompletionTokenLogprob(token='The', bytes=[84, 104, 101], logprob=-0.8941613435745239, top_logprobs=[TopLogprob(token='or', bytes=[111, 114], logprob=-0.8941613435745239), TopLogprob(token='The', bytes=[84, 104, 101], logprob=-0.8941613435745239), TopLogprob(token="You're", bytes=[89, 111, 117, 39, 114, 101], logprob=-2.6441612243652344), TopLogprob(token='This', bytes=[84, 104, 105, 115], logprob=-3.3941612243652344), TopLogprob(token='You', bytes=[89, 111, 117], logprob=-3.3941612243652344)]),
 ChatCompletionTokenLogprob(token=' phrase', bytes=[32, 112, 104, 114, 97, 115, 101], logprob=-0.4791475534439087, top_logprobs=[TopLogprob(token=' phrase', bytes=[32, 112, 104, 114, 97, 115, 101], logprob=-0.4791475534439087), TopLogprob(token=' statement', bytes=[32, 115, 116, 97, 116, 101, 109, 101, 110, 116], logprob=-1.1041475534439087), TopLogprob(token=' line', bytes=[32, 108, 105, 110, 101], logprob=-3.854147434234619), TopLogprob(token=' idea', bytes=[32, 105, 100, 101, 97], logpr

## Vizualize

In [22]:
from IPython.display import HTML, display
import math
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as mcolors


def visualize_confidence(tokens, logprobs, colormap='RdYlGn', darken_factor=0.7):
    """
    Display each token with a background color indicating model confidence.

    Args:
        tokens (List[str]): list of token strings.
        logprobs (List[float]): list of log probability values for each token.
        colormap (str): Matplotlib colormap name for coloring (default 'RdYlGn').
        darken_factor (float): factor in [0,1] to darken colors (default 0.7).
    """
    # Convert logprobs to confidences in [0,1]
    confidences = [math.exp(lp) for lp in logprobs]
    # Normalize confidences to [0,1]
    norm = mcolors.Normalize(vmin=0, vmax=1)
    cmap = cm.get_cmap(colormap)

    html_parts = []
    for tok, conf in zip(tokens, confidences):
        rgba = cmap(norm(conf))
        # Darken the color by mixing with black
        r, g, b, a = rgba
        r *= darken_factor
        g *= darken_factor
        b *= darken_factor
        color = mcolors.to_hex((r, g, b, a))
        # Render white text on dark background
        span = f"<span style='background-color:{color};padding:2px;margin:1px;border-radius:3px;color:white'>{tok}</span>"
        html_parts.append(span)
    html = "".join(html_parts)
    display(HTML(html)) 

In [23]:

# Suppose you have a list of ChatCompletionTokenLogprob objects:
tokens = [t.token for t in completion.choices[0].logprobs.content]
logprobs = [t.logprob for t in completion.choices[0].logprobs.content]

visualize_confidence(tokens, logprobs, colormap='RdYlGn', darken_factor=0.6)

/tmp/ipykernel_771902/3410103644.py:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(colormap)
